# Cognitive Action Probe Testing

Test your trained cognitive action probes on text.

**Key Feature**: Automatically loads the best-performing layer for each action.

In [1]:
import sys
from pathlib import Path
import torch

# Add src/probes to path
sys.path.insert(0, str(Path.cwd() / 'src' / 'probes'))

from best_multi_probe_inference import BestMultiProbeInferenceEngine
from best_probe_loader import print_performance_summary

AMD GPU detected - configuring ROCm environment variables


/home/koalacrown/Desktop/Code/Projects/brije/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. View Probe Performance

In [ ]:
PROBES_BASE_DIR = Path('data/probes_binary')
print_performance_summary(PROBES_BASE_DIR, top_n=10)

## 2. Load Inference Engine

In [ ]:
MODEL_NAME = 'google/gemma-3-4b-it'

print('Loading probes...')
engine = BestMultiProbeInferenceEngine(
    probes_base_dir=PROBES_BASE_DIR,
    model_name=MODEL_NAME
)
print('Ready!')

## 3. Test on Your Text

In [ ]:
text = '''After receiving feedback, I began reconsidering my approach.
I realized I had been making assumptions without fully understanding the constraints.'''

predictions = engine.predict(text, top_k=10, threshold=0.1)

print('Detected Cognitive Actions:')
print('='*70)
for i, pred in enumerate(predictions, 1):
    marker = '✓' if pred.is_active else '○'
    print(f"{marker} {i:2d}. {pred.action_name:30s} {pred.confidence:6.1%}")
    print(f"      (Layer {pred.layer}, AUC: {pred.auc:.3f})")

## 4. Compare Two Texts

In [ ]:
text1 = 'Analyzing the data to identify patterns and trends.'
text2 = 'Brainstorming creative solutions to the problem.'

comparison = engine.compare_texts(text1, text2, top_k=5)

print('TEXT 1:', text1)
print('\nTop actions:')
for action, conf in comparison['text1_top_actions'][:5]:
    print(f'  - {action:30s} {conf:.1%}')

print('\n' + '='*70)
print('TEXT 2:', text2)
print('\nTop actions:')
for action, conf in comparison['text2_top_actions'][:5]:
    print(f'  - {action:30s} {conf:.1%}')

## 5. Batch Processing

In [ ]:
texts = [
    'Comparing different solutions to find the best approach.',
    'Generating creative ideas for the new design.',
    'Evaluating the effectiveness of the strategy.'
]

batch_results = engine.predict_batch(texts, top_k=3, threshold=0.1)

for i, (text, preds) in enumerate(zip(texts, batch_results), 1):
    print(f'\n{i}. {text}')
    for j, pred in enumerate(preds, 1):
        print(f'   {j}. {pred.action_name:30s} {pred.confidence:.1%} (L{pred.layer})')

## 6. View Probe Details

In [ ]:
actions = ['analyzing', 'creating', 'metacognitive_regulation']

print(f"{'Action':<35s} {'Layer':>5s} {'AUC':>8s} {'F1':>8s}")
print('-'*60)
for action in actions:
    info = engine.get_probe_info(action)
    print(f"{info['action']:<35s} {info['layer']:>5d} "
          f"{info['auc']:>8.4f} {info['f1']:>8.4f}")